# **Hands-on**
Reconhecimento de atividades humanas

Esse tutorial tem como objetivo desenvolver um código para realizar o reconhecimento de atividades humanas (Human Activity Recognition - HAR) usando redes neurais e o framework TensorFlow.



## Dataset

O dataset utilizado neste tutorial é o [Motion-sense](https://github.com/mmalekzadeh/motion-sense), o qual inclui dados de séries temporais gerados por sensores de acelerômetro e giroscópio, que fazem as seguintes medições:


*   Pose (roll, pitch, yaw)
*   Aceleração linear
*   Velocidade de rotação



<img src="https://github.com/mmalekzadeh/motion-sense/raw/master/materials/e_map.png"></img>

In [ ]:
from urllib.request import urlretrieve
import os 
!git clone https://github.com/mmalekzadeh/motion-sense.git
!unzip motion-sense/data/A_DeviceMotion_data.zip

Cloning into 'motion-sense'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 612 (delta 14), reused 15 (delta 4), pack-reused 578
Receiving objects: 100% (612/612), 201.78 MiB | 30.89 MiB/s, done.
Resolving deltas: 100% (289/289), done.
Archive:  motion-sense/data/A_DeviceMotion_data.zip
   creating: A_DeviceMotion_data/
   creating: A_DeviceMotion_data/dws_11/
  inflating: A_DeviceMotion_data/dws_11/sub_12.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_1.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_13.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_11.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_2.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_3.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_10.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_14.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_7.csv  
  inflating: A_DeviceMotion_data/dws_11/sub_6.csv  
  inflat

### Labeled Time-Series from data into a Pandas DataFrame

In [3]:
import numpy as np
import pandas as pd

def get_ds_infos():
    """
    Read the file includes data subject information.
    
    Data Columns:
    0: code [1-24]
    1: weight [kg]
    2: height [cm]
    3: age [years]
    4: gender [0:Female, 1:Male]
    
    Returns:
        A pandas DataFrame that contains inforamtion about data subjects' attributes 
    """ 

    dss = pd.read_csv("./motion-sense/data/data_subjects_info.csv")
    print("[INFO] -- Data subjects' information is imported.")
    
    return dss

def set_data_types(data_types=["userAcceleration"]):
    """
    Select the sensors and the mode to shape the final dataset.
    
    Args:
        data_types: A list of sensor data type from this list: [attitude, gravity, rotationRate, userAcceleration] 

    Returns:
        It returns a list of columns to use for creating time-series from files.
    """
    dt_list = []
    for t in data_types:
        if t != "attitude":
            dt_list.append([t+".x",t+".y",t+".z"])
        else:
            dt_list.append([t+".roll", t+".pitch", t+".yaw"])

    return dt_list


def creat_time_series(dt_list, act_labels, trial_codes, mode="mag", labeled=True):
    """
    Args:
        dt_list: A list of columns that shows the type of data we want.
        act_labels: list of activites
        trial_codes: list of trials
        mode: It can be "raw" which means you want raw data
        for every dimention of each data type,
        [attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)].
        or it can be "mag" which means you only want the magnitude for each data type: (x^2+y^2+z^2)^(1/2)
        labeled: True, if we want a labeld dataset. False, if we only want sensor values.

    Returns:
        It returns a time-series of sensor data.
    
    """
    num_data_cols = len(dt_list) if mode == "mag" else len(dt_list*3)

    if labeled:
        dataset = np.zeros((0,num_data_cols+7)) # "7" --> [act, code, weight, height, age, gender, trial] 
    else:
        dataset = np.zeros((0,num_data_cols))
        
    ds_list = get_ds_infos()
    
    print("[INFO] -- Creating Time-Series")
    for sub_id in ds_list["code"]:
        for act_id, act in enumerate(act_labels):
            for trial in trial_codes[act_id]:
                fname = 'A_DeviceMotion_data/'+act+'_'+str(trial)+'/sub_'+str(int(sub_id))+'.csv'
                raw_data = pd.read_csv(fname)
                raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
                vals = np.zeros((len(raw_data), num_data_cols))
                for x_id, axes in enumerate(dt_list):
                    if mode == "mag":
                        vals[:,x_id] = (raw_data[axes]**2).sum(axis=1)**0.5        
                    else:
                        vals[:,x_id*3:(x_id+1)*3] = raw_data[axes].values
                    vals = vals[:,:num_data_cols]
                if labeled:
                    lbls = np.array([[act_id,
                            sub_id-1,
                            ds_list["weight"][sub_id-1],
                            ds_list["height"][sub_id-1],
                            ds_list["age"][sub_id-1],
                            ds_list["gender"][sub_id-1],
                            trial          
                           ]]*len(raw_data))
                    vals = np.concatenate((vals, lbls), axis=1)
                dataset = np.append(dataset,vals, axis=0)
    cols = []
    for axes in dt_list:
        if mode == "raw":
            cols += axes
        else:
            cols += [str(axes[0][:-2])]
            
    if labeled:
        cols += ["act", "id", "weight", "height", "age", "gender", "trial"]
    
    dataset = pd.DataFrame(data=dataset, columns=cols)
    return dataset
#________________________________


ACT_LABELS = ["dws","ups", "wlk", "jog", "std", "sit"]
TRIAL_CODES = {
    ACT_LABELS[0]:[1,2,11],
    ACT_LABELS[1]:[3,4,12],
    ACT_LABELS[2]:[7,8,15],
    ACT_LABELS[3]:[9,16],
    ACT_LABELS[4]:[6,14],
    ACT_LABELS[5]:[5,13]
}


## Here we set parameter to build labeld time-series from dataset of "(A)DeviceMotion_data"
## attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
sdt = ["attitude", "userAcceleration", "gravity", "rotationRate"]
print("[INFO] -- Selected sensor data types: "+str(sdt))    
act_labels = ACT_LABELS# [0:4]
print("[INFO] -- Selected activites: "+str(act_labels))    
trial_codes = [TRIAL_CODES[act] for act in act_labels]
dt_list = set_data_types(sdt)
dataset = creat_time_series(dt_list, act_labels, trial_codes, mode="raw", labeled=True)
print("[INFO] -- Shape of time-Series dataset:"+str(dataset.shape))    
dataset.head()

[INFO] -- Selected sensor data types: ['attitude', 'userAcceleration', 'gravity', 'rotationRate']
[INFO] -- Selected activites: ['dws', 'ups', 'wlk', 'jog', 'std', 'sit']
[INFO] -- Data subjects' information is imported.
[INFO] -- Creating Time-Series
[INFO] -- Shape of time-Series dataset:(1412865, 19)


,attitude.roll,attitude.pitch,attitude.yaw,userAcceleration.x,userAcceleration.y,userAcceleration.z,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,act,id,weight,height,age,gender,trial
0,1.528132,-0.733896,0.696372,0.294894,-0.184493,0.377542,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.0,0.0,102.0,188.0,46.0,1.0,1.0
1,1.527992,-0.716987,0.677762,0.219405,0.035846,0.114866,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.0,0.0,102.0,188.0,46.0,1.0,1.0
2,1.527765,-0.706999,0.670951,0.010714,0.134701,-0.167808,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.0,0.0,102.0,188.0,46.0,1.0,1.0
3,1.516768,-0.704678,0.675735,-0.008389,0.136788,0.094958,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,0.0,0.0,102.0,188.0,46.0,1.0,1.0
4,1.493941,-0.703918,0.672994,0.199441,0.353996,-0.044299,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.0,0.0,102.0,188.0,46.0,1.0,1.0


In [4]:
for i in range(len(ACT_LABELS)):
  lista = np.where(dataset['act'] == i, 1, 0)
  dataset[ACT_LABELS[i]] = np.array(lista)
dataset.head()  

,attitude.roll,attitude.pitch,attitude.yaw,userAcceleration.x,userAcceleration.y,userAcceleration.z,gravity.x,gravity.y,gravity.z,rotationRate.x,...,height,age,gender,trial,dws,ups,wlk,jog,std,sit
0,1.528132,-0.733896,0.696372,0.294894,-0.184493,0.377542,0.741895,0.669768,-0.031672,0.316738,...,188.0,46.0,1.0,1.0,1,0,0,0,0,0
1,1.527992,-0.716987,0.677762,0.219405,0.035846,0.114866,0.753099,0.657116,-0.032255,0.842032,...,188.0,46.0,1.0,1.0,1,0,0,0,0,0
2,1.527765,-0.706999,0.670951,0.010714,0.134701,-0.167808,0.759611,0.649555,-0.032707,-0.138143,...,188.0,46.0,1.0,1.0,1,0,0,0,0,0
3,1.516768,-0.704678,0.675735,-0.008389,0.136788,0.094958,0.760709,0.647788,-0.041140,-0.025005,...,188.0,46.0,1.0,1.0,1,0,0,0,0,0
4,1.493941,-0.703918,0.672994,0.199441,0.353996,-0.044299,0.760062,0.647210,-0.058530,0.114253,...,188.0,46.0,1.0,1.0,1,0,0,0,0,0


#Modelo Centralizado

In [5]:
def minmax_norm(df):
    return (df - df.min()) / ( df.max() - df.min())

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils


features = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'rotationRate.x', 'rotationRate.y', 'rotationRate.z', 'attitude.roll', 'attitude.pitch', 'attitude.yaw']
n_features = len(features)
X = minmax_norm(dataset[features])

In [7]:
# missing data counts
print(f"Dataset Missing Data Counts: {dataset.isna().sum().sum()}")

Dataset Missing Data Counts: 0


In [8]:
dataset.dtypes

attitude.roll         float64
attitude.pitch        float64
attitude.yaw          float64
userAcceleration.x    float64
userAcceleration.y    float64
userAcceleration.z    float64
gravity.x             float64
gravity.y             float64
gravity.z             float64
rotationRate.x        float64
rotationRate.y        float64
rotationRate.z        float64
act                   float64
id                    float64
weight                float64
height                float64
age                   float64
gender                float64
trial                 float64
dws                     int64
ups                     int64
wlk                     int64
jog                     int64
std                     int64
sit                     int64
dtype: object

### Training

In [9]:
# 1 classe
import tensorflow as tf

label = 'dws'
Y = dataset[label]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train, y_train, epochs=10)

2022-03-30 11:56:04.014451: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ale/.cache/pypoetry/virtualenvs/federated_learning_har-X-VryYGz-py3.8/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
35322/35322 [==============================] - 24s 662us/step - loss: 0.2845 - accuracy: 0.9066 - precision: 0.4722 - recall: 0.0061
Epoch 2/10
35322/35322 [==============================] - 24s 673us/step - loss: 0.2804 - accuracy: 0.9067 - precision: 0.4899 - recall: 0.0069
Epoch 3/10
35322/35322 [==============================] - 24s 672us/step - loss: 0.2800 - accuracy: 0.9067 - precision: 0.5073 - recall: 0.0073
Epoch 4/10
35322/35322 [==============================] - 23s 646us/step - loss: 0.2799 - accuracy: 0.9067 - precision: 0.5036 - recall: 0.0073
Epoch 5/10
35322/35322 [==============================] - 23s 660us/step - loss: 0.2799 - accuracy: 0.9067 - precision: 0.5068 - recall: 0.0075
Epoch 6/10
35322/35322 [==============================] - 23s 654us/step - loss: 0.2797 - accuracy: 0.9067 - precision: 0.5058 - recall: 0.0075
Epoch 7/10
35322/35322 [==============================] - 23s 653us/step - loss: 0.2799 - accuracy: 0.9067 - precision: 0.5062 - recall:

In [10]:
from keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense
import tensorflow as tf

Y = dataset[ACT_LABELS]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

def baseline_model():
    model = Sequential()
    model.add(Dense(32, input_dim = n_features, activation='relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(ACT_LABELS), activation = 'softmax')) # Softmax for multi-class classification
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

model = baseline_model()
history = model.fit(X_train, y_train, epochs=30)


Epoch 1/30
33114/33114 [==============================] - 20s 588us/step - loss: 0.9156 - accuracy: 0.6465
Epoch 2/30
33114/33114 [==============================] - 20s 607us/step - loss: 0.6330 - accuracy: 0.7507
Epoch 3/30
33114/33114 [==============================] - 19s 577us/step - loss: 0.6023 - accuracy: 0.7649
Epoch 4/30
33114/33114 [==============================] - 21s 645us/step - loss: 0.5851 - accuracy: 0.7733
Epoch 5/30
33114/33114 [==============================] - 26s 792us/step - loss: 0.5720 - accuracy: 0.7790
Epoch 6/30
33114/33114 [==============================] - 19s 570us/step - loss: 0.5596 - accuracy: 0.7835
Epoch 7/30
33114/33114 [==============================] - 19s 570us/step - loss: 0.5452 - accuracy: 0.7893
Epoch 8/30
33114/33114 [==============================] - 19s 570us/step - loss: 0.5378 - accuracy: 0.7923
Epoch 9/30
33114/33114 [==============================] - 19s 573us/step - loss: 0.5323 - accuracy: 0.7943
Epoch 10/30
33114/33114 [============

In [12]:
from keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense
import tensorflow as tf

Y = dataset[ACT_LABELS]
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Salva os dados de treino 
X_train.to_csv("X_train.csv")
y_train.to_csv("y_train.csv")
X_test.to_csv("X_test.csv")
y_test.to_csv("y_test.csv")
          
def baseline_model():
    model = Sequential()
    model.add(Dense(32, input_dim = n_features, activation='relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(ACT_LABELS), activation = 'softmax')) # Softmax for multi-class classification
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

model = baseline_model()
history = model.fit(X_train, y_train, epochs=100)


Epoch 1/100
33114/33114 [==============================] - 20s 598us/step - loss: 0.8324 - accuracy: 0.6812
Epoch 2/100
33114/33114 [==============================] - 19s 573us/step - loss: 0.6389 - accuracy: 0.7493
Epoch 3/100
33114/33114 [==============================] - 20s 591us/step - loss: 0.6029 - accuracy: 0.7622
Epoch 4/100
33114/33114 [==============================] - 20s 591us/step - loss: 0.5850 - accuracy: 0.7700
Epoch 5/100
33114/33114 [==============================] - 19s 582us/step - loss: 0.5723 - accuracy: 0.7754
Epoch 6/100
33114/33114 [==============================] - 20s 597us/step - loss: 0.5608 - accuracy: 0.7803
Epoch 7/100
33114/33114 [==============================] - 19s 579us/step - loss: 0.5522 - accuracy: 0.7841
Epoch 8/100
33114/33114 [==============================] - 19s 569us/step - loss: 0.5446 - accuracy: 0.7874
Epoch 9/100
33114/33114 [==============================] - 19s 581us/step - loss: 0.5383 - accuracy: 0.7902
Epoch 10/100
33114/33114 [==

In [ ]:
x = [0.6589, 0.7444, 0.7596, 0.7658, 0.7695, 0.7727, 0.7769, 0.7798, 0.7820, 0.7843, 0.7863, 0.7887, 0.7910, 0.7922, 0.7941, 0.7954, 0.7968, 0.7978, 0.7989, 0.7996, 0.8006, 0.8014, 0.8020, 0.8026, 0.8030, 0.8033, 0.8041, 0.8045, 0.8046, 0.8047]
y = [i for i in range(1,31)]
import matplotlib.pyplot as plt
plt.plot(y, x)
plt.xlabel("Épocas")
plt.ylabel("Acurácia")

# Modelo Federado

In [ ]:
pip install git+https://github.com/adap/flower.git

In [ ]:
import flwr

## Cliente Flower

In [ ]:
class Client(flwr.client.NumPyClient):
    def get_parameters(self):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        model.fit(X_train, y_train, epochs=1, batch_size=32, steps_per_epoch=3)
        return model.get_weights(), len(X_train), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy = model.evaluate(X_test, y_test)
        return loss, len(X_test), {"accuracy": accuracy}

In [ ]:
model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=len(ACT_LABELS), weights=None)
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

flwr.client.start_numpy_client("[::]:8081", client=Client())

## Server

In [ ]:
flwr.server.start_server("0.0.0.0:8081", config={"num_rounds": 13})